In [1]:
import requests
import re
from bs4 import BeautifulSoup
import csv
from urllib.request import urlopen
import urllib.parse

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'}
search = input('검색어를 입력하세요 :')
search_url = urllib.parse.quote_plus(search)
pagenum = input('검색할 페이지 수를 입력하세요 :')

search_list = []

for i in range(1, int(pagenum)+1):
#     print(i, '페이지 :')
    url = 'https://www.coupang.com/np/search?q={}&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=4&backgroundColor='.format(search_url, i) 
    res = requests.get(url, headers = headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')

    items = soup.find_all('li', attrs ={'class':re.compile('^search-product')})
    # print(items[0].find('div', attrs={'class':'name'}).get_text())
    for item in items:
        
        # 광고 제품은 제외
        ad_badge = item.find('span', attrs={'class':'ad-badge-text'})
        if ad_badge:
#             print(' <광고 상품 제외합니다.> ')
            continue
        name = item.find('div', attrs={'class':'name'}).get_text() # 제품명
        price = item.find('strong', attrs={'class':'price-value'}).get_text() # 가격
        
        rate = item.find('em', attrs={'class':'rating'}) # 평점
        if rate:
            rate = rate.get_text()
        else:
#             print(' <평점이 없는 상품은 제외합니다.> ')
            continue

        rate_cnt = item.find('span', attrs={'class':'rating-total-count'}) # 평점 수
        if rate_cnt:
            rate_cnt = rate_cnt.get_text()[1:-1] # 예 (56)
    #          print('평점 수', rate_cnt)
        else:
#             print(' <평점 수가 없는 상품은 제외합니다.> ')
            continue
    
        link = item.find('a', attrs={'class':'search-product-link'})['href']
        
        temp = []
        temp.append(name)
        temp.append(price)
        temp.append(rate)
        temp.append(rate_cnt)
        temp.append('https://www.coupang.com' + link)
        search_list.append(temp)
            
        # 평점 수 50개 이상, 평정 4.5 이상 되는 제품들만 조회
        if float(rate) >= 4.5 and int(rate_cnt) >= 50:
            print(f'제품명 : {name}')
            print(f'가격 : {price}')
            print(f'평점 : {rate}점 {rate_cnt}개')
            print('바로가기 : {}'.format('https://www.coupang.com' + link))
            print('-'*50)

print(search_list)                

검색어를 입력하세요 :노트북
검색할 페이지 수를 입력하세요 :3
제품명 : 삼성전자 Plus 2 퓨어화이트 노트북 NT550XDZ-AD1AW (셀러론 6305 39.6cm), 미포함, 256GB, 8GB
가격 : 489,000
평점 : 5.0점 63개
바로가기 : https://www.coupang.com/vp/products/4711649102?itemId=5945569899&vendorItemId=73243525833
--------------------------------------------------
제품명 : 삼성전자 노트북9 METAL NT901X5L 가볍고 슬림한 1.29kg 코어i5 SSD256GB 윈10 탑재, 8GB, SSD 256GB, 포함
가격 : 699,000
평점 : 4.5점 200개
바로가기 : https://www.coupang.com/vp/products/4368907108?itemId=5147093216&vendorItemId=72456426781
--------------------------------------------------
제품명 : 델 Inspiron 15 5505 플래티넘 실버 노트북 DN5505-WH01KR (라이젠5-4500U 39.6cm WIN10 Home), 윈도우 포함, 256GB, 8GB
가격 : 759,000
평점 : 5.0점 99개
바로가기 : https://www.coupang.com/vp/products/1862432591?itemId=3165858124&vendorItemId=71153397923
--------------------------------------------------
제품명 : 삼성전자 퓨어 화이트 노트북 NT350XCR-AD5WA-X16G (i5-10210U 39.6cm), 윈도우 미포함, 256GB, 16GB
가격 : 862,000
평점 : 5.0점 112개
바로가기 : https://www.coupang.com/vp/products/1941837559?itemId

In [69]:
import pandas as pd

with open(f"{search}.csv", 'w', encoding='cp949', newline='') as f: 
    writer = csv.writer(f)
    writer.writerow(['제품명', '가격', '평점', '리뷰 수', '링크'])
    writer.writerows(search_list)

In [70]:
notebook = pd.read_csv('./노트북.csv', encoding='cp949')
notebook.head()

,제품명,가격,평점,리뷰 수,링크
0,삼성전자 갤럭시북 플렉스2 미스틱블랙 노트북 NT950QDZ-G58AB (i5-11...,"2,219,000",5.0,11,https://www.coupang.com/vp/products/4632713564...
1,마이크로소프트 서피스 랩탑 고 플래티넘 노트북 THJ-00021 (i5-1035G1...,"1,172,780",5.0,1,https://www.coupang.com/vp/products/4577278261...
2,삼성전자 퓨어 화이트 노트북 NT350XCR-AD5WA-X16G (i5-10210U...,"862,000",5.0,107,https://www.coupang.com/vp/products/1941837559...
3,델 Inspiron 15 5505 플래티넘 실버 노트북 DN5505-WH01KR (...,"759,000",5.0,97,https://www.coupang.com/vp/products/1862432591...
4,삼성전자 플러스 플래티넘 티탄 노트북 NT350XCR-AD5AS (i5-10210U...,"1,037,000",5.0,135,https://www.coupang.com/vp/products/4698390398...
